In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [1]:
#export
def s3_create_client(s3_bucket):
    """
    Create and set up a connection to S3
    :param s3_bucket:
    :return: the s3 client object.
    """
    access = os.getenv("AWS_ACCESS_KEY_ID")
    secret = os.getenv("AWS_SECRET_ACCESS_KEY")

    session = boto3.Session(
        access,
        secret,
    )
    endpoint_url = os.getenv("AWS_S3_ENDPOINT_URL")

    if endpoint_url is not None:
        s3 = session.resource('s3', endpoint_url=endpoint_url)
    else:
        s3 = session.resource('s3', region_name='eu-west-2')

    bucket = s3.Bucket(s3_bucket)

    if endpoint_url is not None:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret,
            endpoint_url=endpoint_url
        )
    else:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret
        )
    return s3_client, bucket

In [2]:
#export
gb = 1024 ** 3

In [ ]:
#export
def s3_single_upload(in_path, s3_path, s3_bucket):
    """
    put a file into S3 from the local file system.
    :param in_path: a path to a file on the local file system
    :param s3_path: where in S3 to put the file.
    :return: None
    """
    # prep session & creds
    s3_client, bucket = s3_create_client(s3_bucket)

    # Ensure that multipart uploads only happen if the size of a transfer is larger than
    # S3's size limit for non multipart uploads, which is 5 GB. we copy using multipart 
    # at anything over 4gb
    transfer_config = boto3.s3.transfer.TransferConfig(multipart_threshold=2 * gb,
                                                       max_concurrency=10,
                                                       multipart_chunksize=2 * gb,
                                                       use_threads=True)
    s3_client.upload_file(in_path, bucket.name, s3_path)

In [ ]:
#export
def s3_list_objects_paths(s3_bucket, prefix):
    """List of paths only returned, not full object responses"""
    client, bucket = s3_create_client(s3_bucket)
    
    return [e['Key'] for p in client.get_paginator("list_objects_v2").paginate(Bucket=s3_bucket, Prefix=prefix) for e in p['Contents']]


In [ ]:
#export
def s3_list_objects(s3_bucket, prefix):
    # prep session & creds
    client, bucket = s3_create_client(s3_bucket)
    response = client.list_objects_v2(Bucket=s3_bucket, Prefix=prefix)

    return response